# Abnormal Enviornment Detector

In [12]:
import bme680
import time
import numpy as np
from matplotlib import pyplot as plt

sensor = bme680.BME680()

sensor.set_humidity_oversample(bme680.OS_2X)
sensor.set_pressure_oversample(bme680.OS_4X)
sensor.set_temperature_oversample(bme680.OS_8X)
sensor.set_filter(bme680.FILTER_SIZE_3)

sensor.set_gas_status(bme680.ENABLE_GAS_MEAS)
sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

In [15]:
# Finding 'normal' values

# number of minutes to calibrate
cal_time = 5

temp = []
pres = []
hum = []
res = []

for i in range(1, cal_time):
    
    if sensor.get_sensor_data():
        temp.append(sensor.data.temperature)
        pres.append(sensor.data.pressure)
        hum.append(sensor.data.humidity)
        
        if sensor.data.heat_stable:
            res.append(sensor.data.gas_resistance)
            
        else:
            res.append(res[i-1])
            
    else:
        temp.append(temp[i-1])
        pres.append(pres[i-1])
        hum.append(hum[i-1])
    
    time.sleep(60)

avg_temp = np.average(temp)
avg_pres = np.average(pres)
avg_hum = np.average(hum)
avg_res = np.average(res)

print(avg_temp)
print(avg_pres)
print(avg_hum)
print(avg_res)

27.6625
997.9075
61.777499999999996
8465.76946560926


In [18]:
t_alert = False
p_alert = False
h_alert = False
r_alert = False

while True:
    
    if sensor.get_sensor_data():
        
        ########## Temperature ##########
        t = sensor.data.temperature
        t_out = t < avg_temp - 2 or t > avg_temp + 2
        
        if t_out and not t_alert:
            t_alert = True
            print("Temperature is abnormal: " + str(avg_temp - t))
            
        elif not t_out and t_alert:
            t_alert = False
            print("Temperature is normal")
            
        ########## Air Presure ##########
        p = sensor.data.pressure
        p_out = p < avg_pres - 5 or p > avg_pres + 5
        
        if p_out and not p_alert:
            p_alert = True
            print("Pressure is abnormal: " + str(avg_pres - t))
            
        elif not p_out and p_alert:
            p_alert = False
            print("Pressure is normal")
            
        ########## Humidity ##########
        h = sensor.data.humidity
        h_out = h < avg_hum - 5 or h > avg_hum + 5
        
        if h_out and not h_alert:
            h_alert = True
            print("Humidity is abnormal: " + str(avg_hum - h))
            
        elif not h_out and h_alert:
            h_alert = False
            print("Humidity is normal")

        ########## Gas Resistance ##########
        if sensor.data.heat_stable:
            r = sensor.data.gas_resistance
            r_out = r < avg_res - 5 or h > avg_res + 5

            if r_out and not r_alert:
                r_alert = True
                print("Gas Resistance is abnormal: " + str(avg_res - r))

            elif not r_out and r_alert:
                r_alert = False
                print("Gas Resistance is normal")
            
    time.sleep(1)

Humidity is abnormal: -9.323500000000003
Temperature is abnormal: -2.1374999999999993
Temperature is normal
Humidity is normal
Temperature is abnormal: -2.1274999999999977
Humidity is abnormal: -6.224499999999999
Humidity is normal
Temperature is normal
Humidity is abnormal: -6.295499999999997
Temperature is abnormal: -2.0075000000000003
Pressure is abnormal: 967.8075
Pressure is normal
Pressure is abnormal: 965.6475
Pressure is normal
Pressure is abnormal: 964.4275
Pressure is normal
Humidity is normal
Humidity is abnormal: -6.914499999999997
Humidity is normal
Humidity is abnormal: 5.069499999999998
Humidity is normal
Temperature is normal


KeyboardInterrupt: 